<a href="https://colab.research.google.com/github/leemordechai/FLAME/blob/master/FLAME_PAS_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script includes two parts of the process to get PAS' online data.

Part 1 - the first half of this file - requires 4 files (Below). It results in files that should be fed to a separate script to scrape PAS. 
*   coin_finds.csv - from FLAME's PAS download function
*   coin_groups.csv - from FLAME's PAS download function
*   coin_groups_online_filtered.csv - from FLAME-PAS-CoinGroups.ipynb
*   complete_online_finds.csv - from FLAME-PAS.ipynb

Part 2 - the second half of this file - requires a few additional files from the separate script. It results in the final coin_finds and coin_groups files to be uploaded to FLAME.







In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import savefig

%matplotlib inline

from google.colab import files
import pandas as pd
import datetime as dt
import io
import ast

import urllib3
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen, Request
import re
import pandas as pd
import datetime
import time

In [4]:
#@title Uploading FLAME's old coin_finds
_ = files.upload()
flame_finds = pd.read_csv('coin_finds_flame.csv') # manually changed this file's name

In [ ]:
#@title Uploading FLAME's old coin groups
_ = files.upload()
flame_groups = pd.read_csv('coin_groups_flame.csv') # manually changed this file's name

In [ ]:
#@title Uploading finds from online

_ = files.upload()
online_pas_finds = pd.read_csv('complete_online_finds.csv')

In [15]:
#@title Uploading coin groups from online PAS version - this should be run regardless
_ = files.upload()
online_pas_groups = pd.read_csv('coin_groups_online_filtered.csv')

In [8]:
online_ids = []
for entry in flame_finds.name.str.split():
  online_id = int(entry[-1])
  online_ids = online_ids + [online_id]

In [9]:
flame_finds['online_id'] = online_ids
flame_finds.sort_values(by=['online_id']).head(1)
online_pas_finds.sort_values(by=['hoard_id']).head(7)

In [12]:
#@title Checking the difference between the finds from FLAME's PAS dataset and those in PAS
flame_finds_set = set(flame_finds['online_id'])
print(len(flame_finds_set))
online_finds_set = set(online_pas_finds['hoard_id'])
print(len(online_finds_set))
flame_finds_set - online_finds_set

815
1436


{977}

In [13]:
#@title Build a DataFrame
import urllib.request, json 

cols_finds = ['hoard_id', 'unique_id', 'name', 'startDate', 'endDate', 'hoard?', 'excavation?', 'single?', 'num_coins', 'year_found',
		'year_end_found', 'comments', 'lat', 'long', 'certainty', 'owner', 'created', 'imported']
coin_finds = pd.DataFrame(columns=cols_finds)
mismatches = []
unauthorized = []

In [161]:
#@title Get the details of all finds in PAS Online
for search_num in range(64, 65): # change to get the correct numbers 
  try:
    with urllib.request.urlopen('https://finds.org.uk/database/hoards/record/id/' + str(search_num) + '/format/json') as url:
      coin_find_data = json.loads(url.read().decode())
  except:
    print('UNAUTHORIZED: ', str(search_num))
    unauthorized = unauthorized + [search_num]
    continue
  #print(coin_find_data)

  
  hoard_id = int(coin_find_data[1]['id'])
  try:
    name = 'PAS: ' + coin_find_data[1]['knownas'] + ' ' + str(hoard_id)
  except:
    name = 'PAS: Unprovenanced ' + str(hoard_id)
  print(name, end='')
  try:
    year_found = int(coin_find_data[1]['datefound1'][:4])
  except:
    year_found = 'N/A'

  try:
    year_end_found = int(coin_find_data[1]['datefound2'][:4])
  except:
    year_end_found = year_found

  try:
    unique_id = coin_find_data[1]['1']['recordID']
  except:
    try:
      unique_id = coin_find_data[1]['0']['recordID']
    except:
      unique_id = 'need manual'
  created = coin_find_data[1]['created']
  updated = coin_find_data[1]['updated']
  try: 
    comments = (coin_find_data[1]['description'] + ' ******* ' + coin_find_data[1]['notes']).replace('\n','').replace('</p>','').replace('<p>','')
  except:
    comments = (coin_find_data[1]['description']).replace('\n','').replace('</p>','').replace('<p>','')

  try:
    existing_entry = flame_finds[flame_finds['online_id'] == search_num]
    print(' /// ', existing_entry['name'].to_numpy()[0], end='')
    if existing_entry['name'].to_numpy()[0] != name: 
      print(' MISMATCH', end='')
      mismatches = mismatches + [search_num]
    lat = existing_entry['lat'].to_numpy()[0]
    long = existing_entry['long'].to_numpy()[0]
    num_coins = existing_entry['num_coins'].to_numpy()[0]
    certainty = existing_entry['certainty'].to_numpy()[0]
    references = existing_entry['references'].to_numpy()[0]
    tags = existing_entry['Tag_IDs'].to_numpy()[0]
    excav = existing_entry['excavation?'].to_numpy()[0]
    single = existing_entry['single?'].to_numpy()[0]

    coin_finds = coin_finds.append({'hoard_id': int(hoard_id), 'unique_id':unique_id, 'name':name, 'year_found':year_found, 'year_end_found':year_end_found,
                      'comments': comments, 'imported': dt.datetime.now(), 'owner':'PAS UK Finds', 'hoard?':'hoard',
                      'created':created, 'updated':updated, 'lat':lat, 'long':long, 'num_coins':num_coins,
                      'certainty':certainty, 'references':references, 'Tag_IDs':tags, 'excavation?':excav,
                      'single?':single}, ignore_index=True)
    print()

  except:
    print()
    coin_finds = coin_finds.append({'hoard_id': int(hoard_id), 'unique_id':unique_id, 'name':name, 'year_found':year_found, 'year_end_found':year_end_found,
                      'comments': comments, 'imported': dt.datetime.now(), 'owner':'PAS UK Finds', 'hoard?':'hoard',
                      'created':created, 'updated':updated}, ignore_index=True)  

UNAUTHORIZED:  64


In [ ]:
#@title Download results from PAS Online (to be fed to separate file)
coin_finds.to_csv('coin_finds_online_complete.csv')
files.download('coin_finds_online_complete.csv')

<h1>Part 2 - Run from here to upload Finds</h1>

this section does not require any of the cells above other than the first one and the upload of 'coin_groups_online_filtered.csv'. It also requires coin_finds_online_complete and PAS-coordinates (both of which are scraped from PAS in a different file)

In [4]:
#unauthorized2 = [i for i in unauthorized if i < 3296]
mismatches = [12, 14, 15, 22, 43, 52, 53, 60, 94, 141, 142, 155, 177, 180, 193, 198, 207, 222, 223, 235, 239, 241, 250, 274, 277, 310, 312, 354, 382, 392, 393, 407, 482, 484, 485, 542, 543, 548, 563, 593, 600, 604, 605, 608, 617, 628, 629, 634, 648, 664, 676, 701, 710, 729, 732, 738, 742, 775, 790, 803, 814, 823, 843, 845, 867, 870, 877, 878, 881, 908, 919, 937, 962, 976, 977, 979, 996, 1007, 1022, 1034, 1037, 1041, 1043, 1059, 1061, 1112, 1154, 1184, 1193, 1280, 1283, 1284, 1330, 1334, 1336, 1442, 1452, 1460, 1461, 1469, 1470, 1483, 1506, 1518, 1539, 1553, 1554, 1562, 1604, 1615, 1616, 1617, 1620, 1636, 1649, 1650, 1664, 1706, 1722, 1725, 1761, 1814, 1815, 1817, 1819, 1835, 1851, 1852, 1897, 1911, 1927, 1931, 1933, 1943, 1962, 1964, 1973, 1976, 1985, 1986, 2054, 2073, 2092, 2093, 2094, 2110, 2131, 2139, 2148, 2170, 2173, 2174, 2175, 2186, 2210, 2214, 2216, 2223, 2224, 2257, 2270, 2320, 2329, 2359, 2362, 2365, 2374, 2380, 2382, 2412, 2446, 2476, 2493, 2494, 2559, 2665, 2911, 2917, 2919, 2925, 2927, 2928, 2933, 2940, 2941, 2947, 2974, 2978, 2982, 2985, 2995, 3037, 3058, 3080, 3081, 3083, 3115, 3125, 3126, 3130, 3131, 3164, 3166, 3255]
unauthorized = [64, 234, 339, 479, 774, 1079, 1100, 1413, 1870, 1984, 2095, 2101, 2128, 2288, 2355, 3015, 3016, 3017, 3018, 3019, 3020, 3162, 3170, 3266, 3274, 3291, 3292, 3293, 3294, 3295]

In [3]:
_ = files.upload()
total_coin_finds = pd.read_csv('coin_finds_online_complete.csv')

Saving coin_finds_online_complete.csv to coin_finds_online_complete.csv


In [5]:
hoard_ids = list(set(online_pas_groups['hoard_id'])) # hoards_id = all the coin finds with relevant coin groups. 
relevant_finds = total_coin_finds[total_coin_finds['hoard_id'].isin(hoard_ids)] # - returns the actual coin finds 

In [6]:
_ = files.upload()
coordinates = pd.read_csv('PAS-coordinates.csv')
#print(hoard_ids)
#print(coordinates)

Saving PAS-coordinates.csv to PAS-coordinates.csv


In [8]:
#coordinates = coordinates.set_index('Unnamed: 0')
empty_entries = coordinates[coordinates['issues'] == 'no location data']
removing_entries = list(empty_entries['hoard'])
final_finds = relevant_finds[relevant_finds['hoard_id'].isin(removing_entries) == False]

In [10]:
final_finds = final_finds.set_index('hoard_id')
coordinates_clean = coordinates.set_index('hoard')
coordinates_clean = coordinates_clean[coordinates_clean['issues'] != 'no location data']
# set(coordinates_clean.index) - set(final_finds.index) - should be empty

In [11]:
final_finds = final_finds.sort_index()
coordinates_clean = coordinates_clean.sort_index()
final_finds['lat'] = coordinates_clean['lat']
final_finds['long'] = coordinates_clean['long']

coordinates_clean['precision'] = 'highest'
coordinates_clean.loc[coordinates_clean['PAS-precision'] == 'district', 'precision'] = 'lower'
coordinates_clean.loc[coordinates_clean['PAS-precision'] == 'county', 'precision'] = 'lower'
final_finds['certainty'] = coordinates_clean['precision']

In [12]:
num_of_coins_list = []
references_list = []
unique_ids_list = []
hoards_to_search = final_finds.index
for search_num in hoards_to_search:
  URL = 'https://finds.org.uk/database/hoards/record/id/' + str(search_num)
  content = requests.get(URL)
  soup = BeautifulSoup(content.text, 'html.parser')
  all_text = soup.get_text()
  
  unique_id_begin = all_text.find('Unique ID:')
  uniq = all_text[unique_id_begin+11:unique_id_begin+23]
  unique_ids_list = unique_ids_list + [uniq]

  coin_loc = all_text.find('Coins in hoard:')
  if coin_loc > 1:
    newline = all_text[coin_loc:].find('\n')
    num_of_coins = all_text[coin_loc+16:coin_loc + newline]
    num_of_coins_list = num_of_coins_list + [num_of_coins]
  else:
    num_of_coins_list = num_of_coins_list + ['']

  references_loc = all_text.find('References cited\n\n\n\n') + 20
  references_end = all_text[references_loc:].find('Report')
  references = all_text[references_loc:references_loc+references_end].strip().replace("\n",'').replace('        ','\n')
  references_list = references_list + [references]

In [13]:
final_finds['num_coins'] = num_of_coins_list
final_finds['references'] = references_list
final_finds['unique_id'] = unique_ids_list

final_finds['excavation?'] = np.nan
final_finds.loc[final_finds['comments'].str.contains('excav'), 'excavation?'] = 'excav'

In [ ]:
final_finds['certainty'].value_counts()
final_finds = final_finds.drop(['Unnamed: 0'], axis=1)
final_finds = final_finds.rename(columns={'unique_id':'PAS_unique_hoard_id'})
#final_finds.head()

In [27]:
final_finds.to_csv('coin_finds.csv') # this is supposed to be uploaded back to FLAME
files.download('coin_finds.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
filtered_groups = online_pas_groups[online_pas_groups['num_coins'] > 0] # there are some 200 rows of coins without quantity
online_pas_groups['num_coins'].sum() # 300123
filtered_groups = filtered_groups[filtered_groups['revised_end'] > 325]
filtered_groups['num_coins'].sum() #292195 after both filters
print(len(filtered_groups[filtered_groups['num_coins'] < 0])) #all should be 0
print(len(filtered_groups[filtered_groups['revised_end'] < 325]))
print(len(filtered_groups[filtered_groups['revised_end'].isna() == False]))

filtered_groups = filtered_groups.drop(['Unnamed: 0'], axis=1)
filtered_groups = filtered_groups.rename(columns={'hoard_id_long':'PAS_unique_hoard_id'})
filtered_groups['comments'] = np.nan
filtered_groups['Tag_IDs'] = np.nan
filtered_groups.head()

In [ ]:
filtered_groups.to_csv('coin_groups.csv') # this is supposed to be uploaded back to FLAME
files.download('coin_groups.csv')